In [53]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy import stats
import math
import requests
import json

---
# Importing the list of stocks

In [54]:
# Importing the price data of the stocks
df = pd.read_csv('price_data.csv', header=[0,1])
df

SO                                                       \
          Open       High        Low      Close  Adj Close   Volume   
0    72.279999  72.540001  71.250000  71.279999  68.487663  3892700   
1    71.019997  71.639999  70.889999  70.970001  68.189812  3746600   
2    71.360001  71.980003  68.599998  68.620003  65.931862  5397400   
3    68.739998  69.620003  67.510002  67.839996  65.182419  4105500   
4    68.000000  68.529999  67.839996  68.019997  65.355377  3197200   
..         ...        ...        ...        ...        ...      ...   
247  82.970001  83.459999  81.320000  81.660004  81.660004  4028800   
248  81.949997  82.510002  81.820000  82.169998  82.169998  4010100   
249  82.419998  83.300003  82.199997  83.050003  83.050003  3571700   
250  82.639999  83.489998  82.540001  83.379997  83.379997  4032500   
251  82.940002  83.699997  82.290001  83.519997  83.519997  7362300   

            DHI                                      ...         MMM  \
           Open        High         Low       Close  ...         Low   
0    126.610001  128.339996  126.300003  127.919998  ...   92.249161   
1    127.050003  127.400002  124.599998  125.739998  ...   90.434784   
2    124.989998  125.139999  121.790001  123.150002  ...   89.406357   
3    123.680000  126.540001  123.440002  125.709999  ...   87.851173   
4    125.580002  127.800003  125.580002  127.489998  ...   87.700668   
..          ...         ...         ...         ...  ...         ...   
247  172.279999  176.589996  170.330002  172.699997  ...  101.769997   
248  177.559998  180.149994  175.559998  176.940002  ...  112.690002   
249  177.899994  179.800003  176.199997  178.710007  ...  124.129997   
250  179.729996  181.369995  177.240005  180.440002  ...  124.110001   
251  181.360001  185.429993  177.889999  179.929993  ...  125.330002   

                                             KMX                        \
          Close   Adj Close    Volume       Open       High        Low   
0     92.441475   92.441475   3414819  81.839996  84.199997  81.839996   
1     91.061874   91.061874   3810934  83.360001  83.400002  82.480003   
2     89.623749   89.623749   3374514  82.980003  83.610001  81.910004   
3     88.219063   88.219063   3487775  82.860001  84.400002  82.400002   
4     87.984947   87.984947   3001242  83.430000  84.800003  82.589996   
..          ...         ...       ...        ...        ...        ...   
247  103.389999  103.389999   4598100  79.480003  82.900002  79.250000   
248  127.160004  127.160004  32748900  82.610001  83.629997  82.120003   
249  125.160004  125.160004  10816700  83.080002  83.750000  82.449997   
250  126.750000  126.750000   6746500  83.550003  84.489998  82.669998   
251  127.550003  127.550003   6889900  84.500000  86.209999  83.779999   

                                    
         Close  Adj Close   Volume  
0    84.080002  84.080002   995700  
1    83.239998  83.239998  1287300  
2    82.589996  82.589996   648900  
3    83.290001  83.290001  1097600  
4    84.750000  84.750000   764600  
..         ...        ...      ...  
247  81.550003  81.550003  1247800  
248  83.019997  83.019997  1002700  
249  83.260002  83.260002  1022400  
250  84.320000  84.320000  1213400  
251  84.440002  84.440002  1478600  

[252 rows x 3000 columns]

In [55]:
df.columns

MultiIndex([( 'SO',      'Open'),
            ( 'SO',      'High'),
            ( 'SO',       'Low'),
            ( 'SO',     'Close'),
            ( 'SO', 'Adj Close'),
            ( 'SO',    'Volume'),
            ('DHI',      'Open'),
            ('DHI',      'High'),
            ('DHI',       'Low'),
            ('DHI',     'Close'),
            ...
            ('MMM',       'Low'),
            ('MMM',     'Close'),
            ('MMM', 'Adj Close'),
            ('MMM',    'Volume'),
            ('KMX',      'Open'),
            ('KMX',      'High'),
            ('KMX',       'Low'),
            ('KMX',     'Close'),
            ('KMX', 'Adj Close'),
            ('KMX',    'Volume')],
           length=3000)

In [56]:
# Keeping the Adj Close prices only
for column in df.columns:
    if column[1] != 'Adj Close':
        df.drop(column, axis=1, inplace=True)

In [57]:
# Dropping missing values
df.dropna(inplace=True,axis=1)
df

,SO,DHI,MSCI,DVN,CHRW,KR,CSX,REG,CHD,KEY,...,IPGP,KEYS,CL,EVRG,WFC,LRCX,GD,AMGN,MMM,KMX
,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,...,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close
0,68.487663,126.622063,538.231018,51.354408,96.532021,47.092884,32.853542,62.996197,95.006935,11.235960,...,108.360001,161.070007,74.866875,56.717400,44.049267,715.900024,225.339996,232.119995,92.441475,84.080002
1,68.189812,124.464188,537.550720,47.585102,94.638100,47.558578,32.508339,62.948280,95.411133,11.151620,...,111.559998,159.860001,75.746056,56.917580,43.461430,685.880005,226.250000,230.410004,91.061874,83.239998
2,65.931862,121.900467,536.101562,48.627876,95.395660,47.927246,32.459023,63.226177,93.764755,11.282815,...,109.910004,158.570007,74.808266,55.649776,43.849911,693.359985,225.630005,230.699997,89.623749,82.589996
3,65.182419,124.687607,534.602966,48.599174,95.687035,48.014561,31.088072,63.484909,93.291542,11.170362,...,108.879997,158.710007,74.495667,54.620285,43.558544,690.460022,224.009995,243.279999,88.219063,83.290001
4,65.355377,126.453133,543.712402,48.608742,97.124481,48.218300,31.344509,63.858639,94.326691,11.292188,...,111.260002,159.130005,75.120865,54.944386,43.762505,705.359985,227.190002,252.940002,87.984947,84.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,81.660004,172.412048,536.680847,46.279999,89.410004,52.986298,33.599998,64.160004,99.567780,16.129999,...,85.489998,135.889999,96.489998,56.330002,59.247387,884.119995,290.980011,334.299988,103.389999,81.550003
248,82.169998,176.644989,539.014160,46.340000,89.470001,53.940468,34.320000,65.360001,101.093452,16.459999,...,86.589996,138.399994,99.389999,56.750000,59.932156,894.640015,290.399994,334.850006,127.160004,83.019997
249,83.050003,178.412033,541.088257,45.689999,89.250000,53.870892,34.400002,66.239998,101.302856,16.209999,...,87.610001,137.309998,100.879997,57.209999,59.138222,892.340027,289.489990,332.510010,125.160004,83.260002


In [58]:
# Assuming df is your DataFrame
df.columns = df.columns.droplevel(1)
df

,SO,DHI,MSCI,DVN,CHRW,KR,CSX,REG,CHD,KEY,...,IPGP,KEYS,CL,EVRG,WFC,LRCX,GD,AMGN,MMM,KMX
0,68.487663,126.622063,538.231018,51.354408,96.532021,47.092884,32.853542,62.996197,95.006935,11.235960,...,108.360001,161.070007,74.866875,56.717400,44.049267,715.900024,225.339996,232.119995,92.441475,84.080002
1,68.189812,124.464188,537.550720,47.585102,94.638100,47.558578,32.508339,62.948280,95.411133,11.151620,...,111.559998,159.860001,75.746056,56.917580,43.461430,685.880005,226.250000,230.410004,91.061874,83.239998
2,65.931862,121.900467,536.101562,48.627876,95.395660,47.927246,32.459023,63.226177,93.764755,11.282815,...,109.910004,158.570007,74.808266,55.649776,43.849911,693.359985,225.630005,230.699997,89.623749,82.589996
3,65.182419,124.687607,534.602966,48.599174,95.687035,48.014561,31.088072,63.484909,93.291542,11.170362,...,108.879997,158.710007,74.495667,54.620285,43.558544,690.460022,224.009995,243.279999,88.219063,83.290001
4,65.355377,126.453133,543.712402,48.608742,97.124481,48.218300,31.344509,63.858639,94.326691,11.292188,...,111.260002,159.130005,75.120865,54.944386,43.762505,705.359985,227.190002,252.940002,87.984947,84.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,81.660004,172.412048,536.680847,46.279999,89.410004,52.986298,33.599998,64.160004,99.567780,16.129999,...,85.489998,135.889999,96.489998,56.330002,59.247387,884.119995,290.980011,334.299988,103.389999,81.550003
248,82.169998,176.644989,539.014160,46.340000,89.470001,53.940468,34.320000,65.360001,101.093452,16.459999,...,86.589996,138.399994,99.389999,56.750000,59.932156,894.640015,290.399994,334.850006,127.160004,83.019997
249,83.050003,178.412033,541.088257,45.689999,89.250000,53.870892,34.400002,66.239998,101.302856,16.209999,...,87.610001,137.309998,100.879997,57.209999,59.138222,892.340027,289.489990,332.510010,125.160004,83.260002
250,83.379997,180.139145,544.707947,46.169998,89.339996,54.367855,35.020000,68.389999,99.148964,16.250000,...,81.199997,135.949997,100.489998,57.709999,59.555035,854.859985,293.010010,333.279999,126.750000,84.320000


---
# Calculating the 10-day momentum score

In [59]:
tickers = df.columns.tolist()
tickers

['SO',
 'DHI',
 'MSCI',
 'DVN',
 'CHRW',
 'KR',
 'CSX',
 'REG',
 'CHD',
 'KEY',
 'LEG',
 'PG',
 'K',
 'SNPS',
 'HIG',
 'FLS',
 'DUK',
 'TAP',
 'CMI',
 'GOOG',
 'TDY',
 'ACN',
 'KMI',
 'MOS',
 'JKHY',
 'WHR',
 'SPGI',
 'FANG',
 'ESS',
 'DG',
 'FCX',
 'T',
 'VMC',
 'JNJ',
 'DLR',
 'BR',
 'NI',
 'AWK',
 'NTAP',
 'UAL',
 'CRM',
 'FOXA',
 'L',
 'IBM',
 'EXPD',
 'SWKS',
 'PVH',
 'LUV',
 'MCHP',
 'COF',
 'EMR',
 'FTI',
 'CPRT',
 'ANSS',
 'ETN',
 'WELL',
 'PNR',
 'MS',
 'HLT',
 'MLM',
 'DHR',
 'OTIS',
 'MCD',
 'HAL',
 'DVA',
 'NDAQ',
 'SBUX',
 'WEC',
 'SLG',
 'IQV',
 'ALLE',
 'IRM',
 'BBY',
 'VTR',
 'AON',
 'TFX',
 'AIV',
 'NVDA',
 'LLY',
 'CMG',
 'EFX',
 'COO',
 'RJF',
 'PFG',
 'NFLX',
 'FAST',
 'BDX',
 'UNM',
 'UNP',
 'CAT',
 'DLTR',
 'KHC',
 'WDC',
 'TSCO',
 'ECL',
 'ABT',
 'MRO',
 'ZBRA',
 'EA',
 'ADI',
 'EL',
 'SJM',
 'VNO',
 'ADSK',
 'PEP',
 'HOLX',
 'TTWO',
 'BAC',
 'AOS',
 'MTD',
 'ROST',
 'EIX',
 'NTRS',
 'ROP',
 'FFIV',
 'CME',
 'UPS',
 'PSA',
 'PNW',
 'BAX',
 'MHK',
 'ETR',
 'NOW',


In [60]:
# Iterate over the columns to calculate momentum
for ticker in tickers:
    # Create a new column name
    new_column_name = f"{ticker}_pct_change"
    
    # Calculate the percentage change and add it as a new column
    df[new_column_name] = df[ticker].pct_change(periods=10)

# Display the updated DataFrame
df

/var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/ipykernel_29528/1703006700.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = df[ticker].pct_change(periods=10)
/var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/ipykernel_29528/1703006700.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = df[ticker].pct_change(periods=10)
/var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/ipykernel_29528/1703006700.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

,SO,DHI,MSCI,DVN,CHRW,KR,CSX,REG,CHD,KEY,...,IPGP_pct_change,KEYS_pct_change,CL_pct_change,EVRG_pct_change,WFC_pct_change,LRCX_pct_change,GD_pct_change,AMGN_pct_change,MMM_pct_change,KMX_pct_change
0,68.487663,126.622063,538.231018,51.354408,96.532021,47.092884,32.853542,62.996197,95.006935,11.235960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68.189812,124.464188,537.550720,47.585102,94.638100,47.558578,32.508339,62.948280,95.411133,11.151620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,65.931862,121.900467,536.101562,48.627876,95.395660,47.927246,32.459023,63.226177,93.764755,11.282815,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,65.182419,124.687607,534.602966,48.599174,95.687035,48.014561,31.088072,63.484909,93.291542,11.170362,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,65.355377,126.453133,543.712402,48.608742,97.124481,48.218300,31.344509,63.858639,94.326691,11.292188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,81.660004,172.412048,536.680847,46.279999,89.410004,52.986298,33.599998,64.160004,99.567780,16.129999,...,-0.029074,-0.029288,-0.007956,0.030930,-0.007646,-0.165877,0.024037,0.028331,0.001550,0.023084
248,82.169998,176.644989,539.014160,46.340000,89.470001,53.940468,34.320000,65.360001,101.093452,16.459999,...,-0.033917,-0.026107,0.018630,0.030881,0.068093,-0.163747,0.022931,0.012151,0.222222,0.025825
249,83.050003,178.412033,541.088257,45.689999,89.250000,53.870892,34.400002,66.239998,101.302856,16.209999,...,-0.019803,-0.056483,0.042618,0.057095,0.032219,-0.165343,0.014260,0.007148,0.218458,0.015366
250,83.379997,180.139145,544.707947,46.169998,89.339996,54.367855,35.020000,68.389999,99.148964,16.250000,...,-0.104247,-0.090392,0.036988,0.053679,-0.003818,-0.205513,0.004870,0.000420,0.226890,0.003571


In [61]:
# Skipping the first ten rows of data
df2 = df.iloc[10:]
df2

,SO,DHI,MSCI,DVN,CHRW,KR,CSX,REG,CHD,KEY,...,IPGP_pct_change,KEYS_pct_change,CL_pct_change,EVRG_pct_change,WFC_pct_change,LRCX_pct_change,GD_pct_change,AMGN_pct_change,MMM_pct_change,KMX_pct_change
10,65.124771,125.808418,527.615112,47.058922,91.840904,46.797550,29.865065,60.418465,93.122612,9.970861,...,-0.046696,-0.050910,-0.026096,-0.047227,-0.057882,-0.083908,-0.009807,0.146002,-0.085203,0.013202
11,65.441849,121.692169,523.473755,46.417957,91.598099,45.889904,29.776299,59.556034,92.549187,9.867779,...,-0.104338,-0.056675,-0.040237,-0.044716,-0.056536,-0.057722,-0.011757,0.150210,-0.074557,0.018981
12,65.576363,115.552521,514.697083,46.934563,90.646271,45.616634,29.855200,58.482777,91.125496,10.102057,...,-0.099263,-0.053730,-0.033690,-0.027064,-0.059358,-0.064613,-0.014714,0.143693,-0.050285,-0.011381
13,65.828758,116.355919,514.123779,47.824272,92.258545,46.426682,30.446980,58.751091,92.035072,10.148912,...,-0.088905,-0.184866,-0.022423,-0.004968,-0.052397,-0.054341,0.001830,0.076209,-0.052128,-0.009605
14,65.634605,114.997070,518.640686,47.996468,90.830811,46.251007,30.289169,57.888657,91.402328,10.205138,...,-0.109204,-0.185634,-0.040312,-0.016450,-0.057035,-0.051208,-0.012809,0.030679,-0.057398,-0.045428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,81.660004,172.412048,536.680847,46.279999,89.410004,52.986298,33.599998,64.160004,99.567780,16.129999,...,-0.029074,-0.029288,-0.007956,0.030930,-0.007646,-0.165877,0.024037,0.028331,0.001550,0.023084
248,82.169998,176.644989,539.014160,46.340000,89.470001,53.940468,34.320000,65.360001,101.093452,16.459999,...,-0.033917,-0.026107,0.018630,0.030881,0.068093,-0.163747,0.022931,0.012151,0.222222,0.025825
249,83.050003,178.412033,541.088257,45.689999,89.250000,53.870892,34.400002,66.239998,101.302856,16.209999,...,-0.019803,-0.056483,0.042618,0.057095,0.032219,-0.165343,0.014260,0.007148,0.218458,0.015366
250,83.379997,180.139145,544.707947,46.169998,89.339996,54.367855,35.020000,68.389999,99.148964,16.250000,...,-0.104247,-0.090392,0.036988,0.053679,-0.003818,-0.205513,0.004870,0.000420,0.226890,0.003571


In [62]:
df2.iloc[241,0]

83.5199966430664

In [63]:
# Creating a dataframe with the 1Y momentum scores
my_columns = ['Ticker','Price','1Y PCT Change','Number of Shares to Buy']
df3 = pd.DataFrame(columns=my_columns)
df3

,Ticker,Price,1Y PCT Change,Number of Shares to Buy


In [64]:
# Filling the Ticker column
df3['Ticker'] = tickers
df3

,Ticker,Price,1Y PCT Change,Number of Shares to Buy
0,SO,NaN,NaN,NaN
1,DHI,NaN,NaN,NaN
2,MSCI,NaN,NaN,NaN
3,DVN,NaN,NaN,NaN
4,CHRW,NaN,NaN,NaN
...,...,...,...,...
458,LRCX,NaN,NaN,NaN
459,GD,NaN,NaN,NaN
460,AMGN,NaN,NaN,NaN
461,MMM,NaN,NaN,NaN


In [65]:
# Filling the Price column
for i in range(len(tickers)):
    df3.loc[i,'Price'] = df2.iloc[241,i]
df3

,Ticker,Price,1Y PCT Change,Number of Shares to Buy
0,SO,83.519997,NaN,NaN
1,DHI,179.62999,NaN,NaN
2,MSCI,539.223572,NaN,NaN
3,DVN,47.029999,NaN,NaN
4,CHRW,89.050003,NaN,NaN
...,...,...,...,...
458,LRCX,921.23999,NaN,NaN
459,GD,298.709991,NaN,NaN
460,AMGN,332.470001,NaN,NaN
461,MMM,127.550003,NaN,NaN


In [66]:
# Filling the 1Y PCT Change column
for i in range(len(tickers)):
    df3.loc[i,'1Y PCT Change'] = df.iloc[241,(i + len(tickers))]
df3

,Ticker,Price,1Y PCT Change,Number of Shares to Buy
0,SO,83.519997,0.052611,NaN
1,DHI,179.62999,0.164928,NaN
2,MSCI,539.223572,0.025599,NaN
3,DVN,47.029999,0.018422,NaN
4,CHRW,89.050003,0.032734,NaN
...,...,...,...,...
458,LRCX,921.23999,-0.098003,NaN
459,GD,298.709991,0.021623,NaN
460,AMGN,332.470001,0.080124,NaN
461,MMM,127.550003,0.029719,NaN


In [67]:
# Choosing only the top 50 stocks to buy
df3.sort_values(by='1Y PCT Change', ascending=False, inplace=True)
df4 = df3.iloc[0:50]
df4

,Ticker,Price,1Y PCT Change,Number of Shares to Buy
395,VFC,16.959999,0.258768,NaN
120,MHK,161.070007,0.196139,NaN
270,GLW,40.009998,0.194061,NaN
309,BIO,338.359985,0.190955,NaN
1,DHI,179.62999,0.164928,NaN
182,PHM,132.0,0.164562,NaN
415,LEN,176.929993,0.16331,NaN
165,MKTX,223.529999,0.158978,NaN
250,NEM,49.07,0.157276,NaN
330,STT,84.970001,0.155429,NaN


---
# Calculating the number of shares to buy

In [68]:
# Portfolio size input prompt
while True:
    portfolio_size = input('Enter the value of your portfolio:')
    try:
        val = float(portfolio_size)
        if val <= 0:
            print('Please enter a positive non-zero number')
        else:
            print(f"Portfolio value: {val:.2f}")
            break  # Exit the loop if the input is valid
    except ValueError:
        print('Please enter an integer or float value')

Portfolio value: 100000.00


In [69]:
# Position size for each stock
position_size = val / len(df4.index)
print(f'Position Size: {position_size:.2f}')

Position Size: 2000.00


In [70]:
# Calculating the number of shares to buy for each stock
df4['Number of Shares to Buy'] = (position_size / df4['Price']).apply(math.floor)
df4

/var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/ipykernel_29528/3367565602.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Number of Shares to Buy'] = (position_size / df4['Price']).apply(math.floor)


,Ticker,Price,1Y PCT Change,Number of Shares to Buy
395,VFC,16.959999,0.258768,117
120,MHK,161.070007,0.196139,12
270,GLW,40.009998,0.194061,49
309,BIO,338.359985,0.190955,5
1,DHI,179.62999,0.164928,11
182,PHM,132.0,0.164562,15
415,LEN,176.929993,0.16331,11
165,MKTX,223.529999,0.158978,8
250,NEM,49.07,0.157276,40
330,STT,84.970001,0.155429,23
